In [ ]:
# | default_exp routes.datacenter

In [ ]:
# | exporti
from enum import Enum
from typing import Union
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoError as de
import domolibrary.client.DomoAuth as dmda


In [ ]:
#| export
class Datacenter_Enum(Enum):
    ACCOUNT = "ACCOUNT"
    CARD = "CARD"
    DATAFLOW = "DATAFLOW"
    DATASET = "DATASET"
    GROUP = "GROUP"
    PAGE = "PAGE"
    USER = "USER"

class Dataflow_Type_Filter_Enum(Enum):
    ADR = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ADR",
        "name": "ADR",
        "not": False,
    }

    MYSQL = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    REDSHIFT = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    MAGICV2 = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MAGIC",
        "name": "Magic ETL v2",
        "not": False,
    }

    MAGIC = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ETL",
        "name": "Magic ETL",
        "not": False,
    }

In [ ]:
# | export

class Datacenter_Filter_Field_Enum(Enum):
    DATAPROVIDER = "dataprovidername_facet"


def generate_search_datacenter_filter(field,  # use Datacenter_Filter_Field_Enum
                                      value,
                                      is_not: bool = False  # to handle exclusion
                                      ):

    return {
        "filterType": "term",
        "field": field,
        "value": value,
        "not": is_not,
    }


In [ ]:
#| export

def generate_search_datacenter_filter_search_term(search_term):
    # if not "*" in search_term:
    #     search_term = f"*{search_term}*"

    return {"field": "name_sort", "filterType": "wildcard", "query": search_term}


##### sample implementation of generate_search_datacenter_filter_search_term 

In [ ]:
generate_search_datacenter_filter_search_term("hello world*")

{'field': 'name_sort', 'filterType': 'wildcard', 'query': 'hello world*'}

In [ ]:
# | export
def generate_search_datacenter_body(
    search_text : str = None,
    entity_type: Union[str, list] = "DATASET",  # can accept one entity_type or a list of entity_types
    additional_filters_ls: list[dict] = None,
    combineResults: bool = True,
    limit: int = 100,
    offset: int = 0,
):
    filters_ls = [generate_search_datacenter_filter_search_term(search_text)] if search_text else []
    
    if not isinstance(entity_type, list):
        entity_type = [entity_type]

    if additional_filters_ls:
        if not isinstance(additional_filters_ls, list):
            additional_filters_ls = [additional_filters_ls]
        
        filters_ls += additional_filters_ls
    
    return {
        "entities": entity_type,
        "filters": filters_ls or [],
        "combineResults": combineResults,
        "query": "*",
        "count": limit,
        "offset": offset,
    }

#### sample implementation of generate_search_datacenter_body

In [ ]:
from pprint import pprint

sample_search_body = generate_search_datacenter_body(
    search_text = "hello world",
    entity_type= Datacenter_Enum.DATAFLOW.value,
    additional_filters_ls=[Dataflow_Type_Filter_Enum.MAGICV2.value],
)

pprint(sample_search_body)


{'combineResults': True,
 'count': 100,
 'entities': ['DATAFLOW'],
 'filters': [{'field': 'name_sort',
              'filterType': 'wildcard',
              'query': 'hello world'},
             {'field': 'data_flow_type',
              'filterType': 'term',
              'name': 'Magic ETL v2',
              'not': False,
              'value': 'MAGIC'}],
 'offset': 0,
 'query': '*'}


In [ ]:
# | export
def generate_search_datacenter_account_body(
    search_str: str, 
    is_exact_match: bool = True
):
    return {
        "count": 100,
        "offset": 0,
        "combineResults": False,
        "query": search_str if is_exact_match else f"*{search_str}*",
        "filters": [],
        "facetValuesToInclude": [
            "DATAPROVIDERNAME",
            "OWNED_BY_ID",
            "VALID",
            "USED",
            "LAST_MODIFIED_DATE",
        ],
        "queryProfile": "GLOBAL",
        "entityList": [["account"]],
        "sort": {"fieldSorts": [{"field": "display_name_sort", "sortOrder": "ASC"}]},
    }

In [ ]:
# | export
class SearchDatacenter_NoResultsFound(de.DomoError):
    def __init__(self, body, domo_instance):
        super().__init__(message = body, domo_instance = domo_instance)

async def search_datacenter(
    auth: dmda.DomoAuth,
    maximum: int = None,
    
    body: dict = None, # either pass a body or generate a body in the function using search_text, entity_type, and additional_filters parameters
    
    search_text = None,
    entity_type: Union[str, list] = "dataset", # can accept one value or a list of values
    additional_filters_ls = None,
    
    arr_fn: callable = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    limit = 100  # api enforced limit

    if not body:
        body = generate_search_datacenter_body(
            entity_type=entity_type,
            additional_filters_ls= additional_filters_ls,
            search_text=search_text,
            combineResults=False,
            limit = limit 
        )

    if not arr_fn:
        def arr_fn(res):
            return res.response.get("searchObjects")

    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"

    res = await gd.looper(
        auth=auth,
        session=session,
        url=url,
        body=body,
        offset_params_in_body=True,
        offset_params={"offset": "offset", "limit": "count"},
        arr_fn=arr_fn,
        method="POST",
        loop_until_end= True if not maximum else False,
        maximum = maximum,
        limit=limit,
        debug_api=debug_api,
    )

    if res.is_success and len(res.response) == 0:
        raise SearchDatacenter_NoResultsFound(body = body, domo_instance = auth.domo_instance)

    return res

#### sample implementation of generate_search body and search datacenter


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)


additional_filters_ls = [
    generate_search_datacenter_filter("dataprovidername_facet", "Jupyter Data")]


res = await search_datacenter(auth=token_auth,
                              search_text="*kb*",
                              entity_type=Datacenter_Enum.DATASET.value,
                              additional_filters_ls=additional_filters_ls
                              )

pd.DataFrame(res.response[0:5])


,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,pdpEnabled,used,owners,ownersLocalized,cloudName,cloudId,winnerText,systemTagFields,ownedByType,customer
0,dataset,04c1574e-c8be-4721-9846-c6ffa491144b,"{'indexName': None, 'databaseId': '04c1574e-c8...",1668379680000,1680616622000,1680617077900,{},English,False,154.36414,...,False,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson1', 'count': 1}",Domo,domo,domo_kbs,[system_tags],USER,mmmm-0012-0200


In [ ]:
# | export
async def get_lineage_upstream(
    auth: dmda.DomoAuth,
    entity_type: str,
    entity_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/lineage/{entity_type}/{entity_id}"

    params = {"traverseDown": "false"}

    return await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        params=params,
        session=session,
        debug_api=debug_api,
    )

#### sample implementation of get_lineage_upstream

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance='domo-community',
    domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

res = await get_lineage_upstream(auth=token_auth,
                                 entity_type = Datacenter_Enum.DATAFLOW.value,
                                 entity_id=4

                                 )

pd.DataFrame([ obj for obj in res.response.values()])

,type,id,descendantCounts,ancestorCounts,complete,children,parents
0,DATA_SOURCE,45a7a24e-c738-44f9-9019-00d5a33613ae,{},"{'DATA_SOURCE': 2, 'DATAFLOW': 2}",True,"[{'type': 'DATAFLOW', 'id': '4', 'complete': T...","[{'type': 'DATAFLOW', 'id': '3', 'complete': T..."
1,DATAFLOW,4,{},"{'DATA_SOURCE': 3, 'DATAFLOW': 2}",True,[],"[{'type': 'DATA_SOURCE', 'id': '45a7a24e-c738-..."
2,DATAFLOW,2,{},{'DATA_SOURCE': 1},True,"[{'type': 'DATA_SOURCE', 'id': '0647a9ed-1a8e-...","[{'type': 'DATA_SOURCE', 'id': '241025d7-3cca-..."
3,DATA_SOURCE,0647a9ed-1a8e-420e-bd66-34f313a18595,{},"{'DATA_SOURCE': 1, 'DATAFLOW': 1}",True,"[{'type': 'DATAFLOW', 'id': '3', 'complete': T...","[{'type': 'DATAFLOW', 'id': '2', 'complete': T..."
4,DATAFLOW,3,{},"{'DATA_SOURCE': 2, 'DATAFLOW': 1}",True,"[{'type': 'DATA_SOURCE', 'id': '45a7a24e-c738-...","[{'type': 'DATA_SOURCE', 'id': '0647a9ed-1a8e-..."
5,DATA_SOURCE,241025d7-3cca-4369-b7c0-b3264277c0e1,{},{},True,"[{'type': 'DATAFLOW', 'id': '2', 'complete': T...",[]


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
